In [3]:
import os
import torch
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import numpy as np
import os

import scipy
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.metrics import f1_score

import torch.nn.utils as torch_utils
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic,ConstantKernel, Matern

from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [8]:
from ssda import data_path as data_dir
data_dir = os.path.join(data_dir,"raw","porous")
data_path = os.path.join(data_dir,"Charite_CortBS_Simulations.mat")

In [5]:
#data_dir = "./drive/MyDrive/Work/Potsdam/Charite/"
#data_path = os.path.join(data_dir,"./Charite_CortBS_Simulations.mat")

In [9]:
mat = scipy.io.loadmat(data_path)

In [12]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'BSC_5', 'BSC_6', 'BSC_7', 'BSC_8', 'NDS', 'PoDmD', 'PoDmD_ks', 'PoDm_mean', 'PoDm_median', 'PoDm_peak', 'PoDm_q10', 'PoDm_q90', 'PoDm_std', 'PoDm_vec', 'PodDm_gamma_a', 'PodDm_gamma_b', 'absorption', 'att_0', 'att_5', 'att_6', 'att_7', 'att_8', 'att_f', 'f', 'po_dm_base', 'po_dm_gradient', 'pores_size', 'porosity_base', 'z'])

In [13]:
def pick_data(mat,target_name="PoDm_mean"):
    X_ = mat['NDS']
    Y = mat[target_name].T
    return  X_,Y

def inpute_data_cut(X_,cut_size_x = 5,cut_size_y = 5):
    # Calculate the mean of each column
    X_reduced = X_[:,cut_size_x:-cut_size_x,cut_size_y:-cut_size_y]
    assert not np.isnan(X_reduced).any()       
    return X_reduced

def inpute_data_mean(X_):
    # Calculate the mean of each column
    col_mean = np.nanmean(X_, axis=0)
    row_mean = np.nanmean(X_, axis=1)
    # Replace NaN values with the mean of the column
    X_[np.isnan(X_)] = np.take(col_mean, np.isnan(X_).nonzero()[1])
    X_[np.isnan(X_)] = np.take(row_mean, np.isnan(X_).nonzero()[1])
    return X_

def dimensionality_reduction(X_,none=False):
    datasize = X_.shape[0]
    if none:
        X_reduced = X_.reshape(datasize,-1)
    else:
        # Perform dimensionality reduction using PCA
        pca = PCA(n_components=100)
        X_reduced = pca.fit_transform(X_.reshape(datasize,-1))
    return X_reduced

def normalize(X_reduced,YF):
    # Calculate mean and standard deviation of the input X
    X_mean = np.mean(X_reduced, axis=0)
    X_std = np.std(X_reduced, axis=0)
    # Normalize the input X
    X_normalized = (X_reduced - X_mean) / X_std
    # Calculate mean and standard deviation of the output Y
    Y_mean = np.mean(YF, axis=0)
    Y_std = np.std(YF, axis=0)
    # Normalize the output Y
    Y_normalized = (YF - Y_mean) / Y_std
    return X_normalized, Y_normalized

def permute_data(X,Y):
    permutation = torch.randperm(torch.Tensor(X).size()[0])
    X = torch.Tensor(X)[permutation].numpy()
    Y = torch.Tensor(Y)[permutation].numpy()
    return X,Y

def one_experiment(X_reduced,Y,kernel,target_name="PoDm_mean",svm=False):
    X_reduced, Y = permute_data(X_reduced,Y)
    X_train, X_test, Y_train, Y_test = train_test_split(X_reduced, Y, test_size=0.25)
    if target_name == "absortion":
        if svm:
            gpr = svm.SVC()
        else:
            gpr = GaussianProcessClassifier(kernel=kernel)
    else:
        gpr = GaussianProcessRegressor(kernel=kernel)
        
    gpr.fit(X_train, Y_train)
    Y_pred = gpr.predict(X_test)
    if target_name == "absortion":
        r2 = f1_score(Y_test, Y_pred,average='weighted')
    else:
        r2 = r2_score(Y_test, Y_pred)
    return r2

In [91]:
#target_name = "PoDm_mean"
#target_name = "PoDm_std"
#target_name = "PoDm_q90"
#target_name = "PoDm_q10"
#target_name = "absorption"
#kernel_str = "rq"

svm=False
for target_name in ["PoDm_mean","PoDm_std","PoDm_q90","PoDm_q10","absorption"]:
    for kernel_str in ["rq","m"]:
        #===========================
        # SELECT KERNEL
        #===========================
        print("Data {0} Kernel {1}".format(target_name,kernel_str))
        if kernel_str == "rq":
            kernel = RationalQuadratic(length_scale=1.0, 
                                       length_scale_bounds=(1e-9,1e9), 
                                       alpha=1.5, 
                                       alpha_bounds=(1e-6,1e2))
        elif kernel_str == "m":
            kernel = Matern(length_scale=1.0, 
                            length_scale_bounds=(1e-9,1e9))
        elif kernel_str == "rbf":
            kernel = RBF(length_scale=1.0, 
                         length_scale_bounds=(1e-9,1e9))
        #==============================
        # PICK DATA
        #=============================
        X_,Y = pick_data(mat,target_name=target_name)
        if target_name == "absortion":
            number_of_data_points = len(set(Y.squeeze()))
            float_values_to_cat = dict(zip(set(Y.squeeze()),range(number_of_data_points)))
            for i,v in enumerate(Y):
                Y[i,0] = int(float_values_to_cat[Y[i,0]])

        X_ = X_.transpose(2,0,1)
        X_reduced = inpute_data_cut(X_,cut_size_x=5,cut_size_y = 5)
        if target_name != "absortion":
            X_reduced,Y = normalize(X_reduced,Y)
        X_reduced = dimensionality_reduction(X_reduced,False)
                       
        print(X_reduced.shape)
        #==============================
        # EXPERIMENTS
        #=============================
        R2  = []
        number_of_experiments = 20
        for a in range(0,number_of_experiments):
            r2 = one_experiment(X_reduced,Y,kernel,target_name,svm=svm)
            R2.append(r2)
            if a % 10 == 0:
                print("Experiment {0}".format(a))
        R2 = np.asarray(R2)

        experiment_results = {"kernel":kernel,
                              "svm":svm,
                              "r2 mean":R2.mean(),
                              "r2 std":R2.std(),
                              "target_name":target_name}
        
        RESULTS.append(experiment_results)
        print(experiment_results)

Data PoDm_mean Kernel rq
(780, 100)
Experiment 0
Experiment 10
{'kernel': RationalQuadratic(alpha=1.5, length_scale=1), 'svm': False, 'r2 mean': 0.8179355138771598, 'r2 std': 0.01640117262321306, 'target_name': 'PoDm_mean'}
Data PoDm_mean Kernel m
(780, 100)
Experiment 0


C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or sca

Experiment 10


C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


{'kernel': Matern(length_scale=1, nu=1.5), 'svm': False, 'r2 mean': 0.8207114299498327, 'r2 std': 0.020318888269418613, 'target_name': 'PoDm_mean'}
Data PoDm_std Kernel rq
(780, 100)
Experiment 0
Experiment 10
{'kernel': RationalQuadratic(alpha=1.5, length_scale=1), 'svm': False, 'r2 mean': 0.8076966497652676, 'r2 std': 0.01733804646687628, 'target_name': 'PoDm_std'}
Data PoDm_std Kernel m
(780, 100)
Experiment 0


...

Experiment 0


C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Experiment 10


C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\cesar\anaconda3\envs\discrete-diffusion\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


{'kernel': Matern(length_scale=1, nu=1.5), 'svm': False, 'r2 mean': 0.9970168350375473, 'r2 std': 0.0005951149195120197, 'target_name': 'absorption'}


In [93]:
RESULTS

[{'kernel': RationalQuadratic(alpha=1.5, length_scale=1),
  'svm': False,
  'r2 mean': 0.8219790381767333,
  'r2 std': 0.017696652839870496,
  'target_name': 'PoDm_mean'},
 {'kernel': RationalQuadratic(alpha=1.5, length_scale=1),
  'svm': False,
  'r2 mean': 0.7983270342306585,
  'r2 std': 0.019626185698025978,
  'target_name': 'PoDm_std'},
 {'kernel': Matern(length_scale=1, nu=1.5),
  'svm': False,
  'r2 mean': 0.8044986244044784,
  'r2 std': 0.016324958963729264,
  'target_name': 'PoDm_std'},
 {'kernel': RBF(length_scale=1),
  'svm': False,
  'r2 mean': -0.0043856328311607466,
  'r2 std': 0.005534543531034067,
  'target_name': 'PoDm_std'},
 {'kernel': RationalQuadratic(alpha=1.5, length_scale=1),
  'svm': False,
  'r2 mean': 0.8044557211009249,
  'r2 std': 0.02846775654563559,
  'target_name': 'PoDm_q90'},
 {'kernel': Matern(length_scale=1, nu=1.5),
  'svm': False,
  'r2 mean': 0.8017413938280376,
  'r2 std': 0.020484743138997806,
  'target_name': 'PoDm_q90'},
 {'kernel': RBF(length_

In [80]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA()
X_1 = inpute_data_cut(X_,cut_size_x=5,cut_size_y = 5)
X_1.reshape(114,-1)

#pca.fit(X_1)

#plt.plot(np.cumsum(pca.explained_variance_ratio_))
#plt.xlabel('Number of components')
#plt.ylabel('Cumulative explained variance')
#plt.show()

(114, 25, 770)

In [77]:
#RESULTS.append(experiment_results)

In [87]:
Y.shape

(780, 1)